**Problem 2: Forest Fire Classification**

The dataset is designed for binary classification of Fire and No-Fire detection in the forests landscape. You should do this classification using logistic regression. Do the following steps:

a) Pre-processing: The size of the images may be different. Resize them. Then normalize the images. Determine the target of each image. (We suggest using glob and OpenCV libraries).

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import glob
import cv2
import numpy as np

dataset_path = '/content/drive/My Drive/fire_dataset'
img_size = (64, 64)

# get the file paths of all the fire and non-fire images in the dataset directory
fire_image_files = glob.glob(f'{dataset_path}/fire_images/fire.*.png')
non_fire_image_files = glob.glob(f'{dataset_path}/non_fire_images/non_fire.*.png')

# load the images and their labels
images = []
labels = []

for img_file in fire_image_files:
    # load the image using OpenCV
    img = cv2.imread(img_file)
    # resize the image to the desired size
    img = cv2.resize(img, img_size)
    # normalize the pixel values to be between 0 and 1
    img = img / 255.0
    # add the image to the list of images
    images.append(img)
    # add the label to the list of labels
    labels.append(1)

for img_file in non_fire_image_files:
    # load the image using OpenCV
    img = cv2.imread(img_file)
    # resize the image to the desired size
    img = cv2.resize(img, img_size)
    # normalize the pixel values to be between 0 and 1
    img = img / 255.0
    # add the image to the list of images
    images.append(img)
    # add the label to the list of labels
    labels.append(0)

# convert the images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)


b) Split the data into train and test.

In [36]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets with a 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# reshape X_train and X_test to a 2D array

X_train = X_train.reshape(X_train.shape[0], -1)

X_test = X_test.reshape(X_test.shape[0], -1)

# Print the shapes of the resulting train and test sets
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Train set shape: (798, 12288) (798,)
Test set shape: (200, 12288) (200,)


c) Run logistic regression on train data.

In [37]:
from sklearn.linear_model import LogisticRegression

# create a logistic regression model object
model = LogisticRegression()

# train the model on the training data
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

d) Report accuracy and confusion matrix for test data.

In [38]:
from sklearn.metrics import accuracy_score, confusion_matrix

# use the trained model to make predictions on the test data
y_pred = model.predict(X_test)

# compute the accuracy score of the model on the test data
acc_score = accuracy_score(y_test, y_pred)
print('Accuracy:', acc_score)

# compute the confusion matrix of the model on the test data
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)


Accuracy: 0.925
Confusion Matrix:
[[ 46   8]
 [  7 139]]


e) Find the best probability threshold for the training data and report the accuracy and confusion matrix again. Did the results improve?

In [39]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

probs = model.predict_proba(X_train)
thresholds = np.linspace(0, 1, 101)

best_acc = 0
best_threshold = 0

for threshold in thresholds:
    y_pred = (probs[:,1] > threshold).astype(int)
    acc = accuracy_score(y_train, y_pred)
    if acc > best_acc:
        best_acc = acc
        best_threshold = threshold

probs_test = model.predict_proba(X_test)
y_pred_test = (probs_test[:,1] > best_threshold).astype(int)

acc_test = accuracy_score(y_test, y_pred_test)
cm_test = confusion_matrix(y_test, y_pred_test)

print("Test accuracy:", acc_test)
print("Test confusion matrix:\n", cm_test)


Test accuracy: 0.9
Test confusion matrix:
 [[ 35  19]
 [  1 145]]


f)Save the best model. import this model to another file (e.g., classifier.py). Download some forest images (fire and non-fire). feed this image to your classifier and make a prediction. Finally, show the label of each image with its probability in the photo. If the photo was fire, with red color, otherwise green color (similar to below).

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from joblib import dump


# define the hyperparameter grid with 
param_grid = {
    'C': [0.1, 1],
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga']
}

# create a logistic regression object
model = LogisticRegression()

# perform grid search over the hyperparameter grid
grid = GridSearchCV(model, param_grid, cv=5)
grid.fit(X_train, y_train)

# get the best model
best_model = grid.best_estimator_

# save the best model to a file
dump(best_model, 'best_model.joblib')


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

['best_model.joblib']